In [1]:
from datasets import load_dataset, load_metric, Audio, Dataset
import os
import torchaudio
from tqdm.auto import tqdm

In [ ]:
# %%bash 
# wget https://www.openslr.org/resources/42/km_kh_male.zip
# unzip km_kh_male.zip

--2022-02-03 05:13:35--  https://www.openslr.org/resources/42/km_kh_male.zip
Resolving www.openslr.org (www.openslr.org)... 46.101.158.64
Connecting to www.openslr.org (www.openslr.org)|46.101.158.64|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 866086951 (826M) [application/zip]
Saving to: ‘km_kh_male.zip’

     0K .......... .......... .......... .......... ..........  0%  264K 53m29s
    50K .......... .......... .......... .......... ..........  0%  525K 40m9s
   100K .......... .......... .......... .......... ..........  0%  523K 35m45s
   150K .......... .......... .......... .......... ..........  0%  523K 33m33s
   200K .......... .......... .......... .......... ..........  0%  264K 37m31s
   250K .......... .......... .......... .......... ..........  0%  523K 35m45s
   300K .......... .......... .......... .......... ..........  0%  526K 34m28s
   350K .......... .......... .......... .......... ..........  0%  524K 33m31s
   400K .......... ....

Archive:  km_kh_male.zip
   creating: km_kh_male/
  inflating: km_kh_male/line_index.tsv  
   creating: km_kh_male/wavs/
  inflating: km_kh_male/wavs/khm_1161_3945210975.wav  
  inflating: km_kh_male/wavs/khm_1161_3840820726.wav  
  inflating: km_kh_male/wavs/khm_1161_3632689663.wav  
  inflating: km_kh_male/wavs/khm_1161_3514535297.wav  
  inflating: km_kh_male/wavs/khm_1161_3445330166.wav  
  inflating: km_kh_male/wavs/khm_1161_3170598248.wav  
  inflating: km_kh_male/wavs/khm_1161_3030414325.wav  
  inflating: km_kh_male/wavs/khm_1161_2858870182.wav  
  inflating: km_kh_male/wavs/khm_1161_2835582962.wav  
  inflating: km_kh_male/wavs/khm_1161_2662937440.wav  
  inflating: km_kh_male/wavs/khm_1161_2449542221.wav  
  inflating: km_kh_male/wavs/khm_1161_2294049689.wav  
  inflating: km_kh_male/wavs/khm_1161_2136371765.wav  
  inflating: km_kh_male/wavs/khm_1161_2092267195.wav  
  inflating: km_kh_male/wavs/khm_1161_2061040949.wav  
  inflating: km_kh_male/wavs/khm_1161_2054764716.wav  

### Load KH Data

In [6]:
# from sklearn.model_selection import train_test_split
# import pandas as pd
# from datasets import load_dataset
 
# colnames=['path','drop','text'] 
# df  = pd.read_csv('km_kh_male/line_index.tsv',sep='\t',header=None,names=colnames)
# df['path'] = '/workspace/xls-r-300m-km/km_kh_male/wavs/'+df['path'] +'.wav'

# train_valid, test = train_test_split(df, test_size=0.1)
# train, valid = train_test_split(train_valid, test_size=0.1)

# train.to_csv('./km_kh_male/line_index_train.csv')
# valid.to_csv('./km_kh_male/line_index_valid.csv')
# test.to_csv('./km_kh_male/line_index_test.csv')

In [3]:
common_voice_train  = load_dataset('csv', data_files='km_kh_male/line_index_train_merge.csv', split= 'train')
common_voice_valid  = load_dataset('csv', data_files='km_kh_male/line_index_valid_short.csv', split = 'train')

Using custom data configuration default-ac5c5990503e1315


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration default-14880bfd1be4a95b


Dataset csv downloaded and prepared to /workspace/.cache/huggingface/datasets/csv/default-ac5c5990503e1315/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /workspace/.cache/huggingface/datasets/csv/default-14880bfd1be4a95b/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e. Subsequent calls will reuse this data.


In [4]:
common_voice_train = (common_voice_train
                      .remove_columns(["Unnamed: 0", "drop"])
                      .rename_column('text', 'sentence'))
common_voice_valid  = (common_voice_valid
                      .remove_columns(["Unnamed: 0", "drop"])
                      .rename_column('text', 'sentence'))

In [5]:
common_voice_train[0]

{'path': '/workspace/xls-r-300m-km/km_kh_male/wavs/khm_1443_4015603856.wav',
 'sentence': 'ទេសចរណ៍ នៅ ខេត្ត ព្រះ សីហនុ នា រដូវ បុណ្យ ភ្ជុំ បិណ្ឌ នេះ មាន ការ កើន ឡើង យ៉ាង ខ្លាំង'}

### Build Character

In [6]:
vocab_train = []
vocab_test  = []

for batch in tqdm(common_voice_train):
    sentence = batch['sentence']
    vocab_train.extend(list(set(list(sentence))))
    
for batch in tqdm(common_voice_valid):
    sentence = batch['sentence']
    vocab_test.extend(list(set(list(sentence))))

  0%|          | 0/2586 [00:00<?, ?it/s]

  0%|          | 0/29 [00:00<?, ?it/s]

In [7]:
vocab_list = list(set(vocab_train) | set(vocab_test))
vocab_dict = {v: k for k, v in enumerate(sorted(vocab_list))}

In [8]:
print(vocab_dict)

{' ': 0, 'ក': 1, 'ខ': 2, 'គ': 3, 'ឃ': 4, 'ង': 5, 'ច': 6, 'ឆ': 7, 'ជ': 8, 'ឈ': 9, 'ញ': 10, 'ដ': 11, 'ឋ': 12, 'ឌ': 13, 'ឍ': 14, 'ណ': 15, 'ត': 16, 'ថ': 17, 'ទ': 18, 'ធ': 19, 'ន': 20, 'ប': 21, 'ផ': 22, 'ព': 23, 'ភ': 24, 'ម': 25, 'យ': 26, 'រ': 27, 'ល': 28, 'វ': 29, 'ស': 30, 'ហ': 31, 'ឡ': 32, 'អ': 33, 'ឥ': 34, 'ឧ': 35, 'ឪ': 36, 'ឫ': 37, 'ឬ': 38, 'ឭ': 39, 'ឮ': 40, 'ឯ': 41, 'ឱ': 42, 'ា': 43, 'ិ': 44, 'ី': 45, 'ឹ': 46, 'ឺ': 47, 'ុ': 48, 'ូ': 49, 'ួ': 50, 'ើ': 51, 'ឿ': 52, 'ៀ': 53, 'េ': 54, 'ែ': 55, 'ៃ': 56, 'ោ': 57, 'ៅ': 58, 'ំ': 59, 'ះ': 60, 'ៈ': 61, '៉': 62, '៊': 63, '់': 64, '៌': 65, '៍': 66, '៎': 67, '៏': 68, '័': 69, '្': 70}


In [9]:
# make the space more intuitive to understand
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

73

In [10]:
print(vocab_dict)

{'ក': 1, 'ខ': 2, 'គ': 3, 'ឃ': 4, 'ង': 5, 'ច': 6, 'ឆ': 7, 'ជ': 8, 'ឈ': 9, 'ញ': 10, 'ដ': 11, 'ឋ': 12, 'ឌ': 13, 'ឍ': 14, 'ណ': 15, 'ត': 16, 'ថ': 17, 'ទ': 18, 'ធ': 19, 'ន': 20, 'ប': 21, 'ផ': 22, 'ព': 23, 'ភ': 24, 'ម': 25, 'យ': 26, 'រ': 27, 'ល': 28, 'វ': 29, 'ស': 30, 'ហ': 31, 'ឡ': 32, 'អ': 33, 'ឥ': 34, 'ឧ': 35, 'ឪ': 36, 'ឫ': 37, 'ឬ': 38, 'ឭ': 39, 'ឮ': 40, 'ឯ': 41, 'ឱ': 42, 'ា': 43, 'ិ': 44, 'ី': 45, 'ឹ': 46, 'ឺ': 47, 'ុ': 48, 'ូ': 49, 'ួ': 50, 'ើ': 51, 'ឿ': 52, 'ៀ': 53, 'េ': 54, 'ែ': 55, 'ៃ': 56, 'ោ': 57, 'ៅ': 58, 'ំ': 59, 'ះ': 60, 'ៈ': 61, '៉': 62, '៊': 63, '់': 64, '៌': 65, '៍': 66, '៎': 67, '៏': 68, '័': 69, '្': 70, '|': 0, '[UNK]': 71, '[PAD]': 72}


In [11]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

# Tokenizer

In [12]:
from transformers import Wav2Vec2CTCTokenizer
from transformers import Wav2Vec2FeatureExtractor
from transformers import Wav2Vec2Processor

In [13]:
tokenizer = Wav2Vec2CTCTokenizer.from_pretrained("./", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")  # './' load vocab.json in the current directory
feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)  
processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [14]:
common_voice_train = common_voice_train.cast_column("path", Audio(sampling_rate=16_000)).rename_column('path', 'audio')
common_voice_valid  = common_voice_valid.cast_column("path", Audio(sampling_rate=16_000)).rename_column('path', 'audio')

In [15]:
common_voice_train[0]

{'audio': {'path': '/workspace/xls-r-300m-km/km_kh_male/wavs/khm_1443_4015603856.wav',
  'array': array([-1.3359112e-06,  1.5759380e-06, -2.0205737e-06, ...,
         -7.6091878e-06,  9.0511895e-07,  0.0000000e+00], dtype=float32),
  'sampling_rate': 16000},
 'sentence': 'ទេសចរណ៍ នៅ ខេត្ត ព្រះ សីហនុ នា រដូវ បុណ្យ ភ្ជុំ បិណ្ឌ នេះ មាន ការ កើន ឡើង យ៉ាង ខ្លាំង'}

In [16]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(common_voice_train)-1)

print("Target text:", common_voice_train[rand_int]["sentence"])
print("Input array shape:", common_voice_train[rand_int]["audio"]["array"].shape)
print("Sampling rate:", common_voice_train[rand_int]["audio"]["sampling_rate"])
ipd.Audio(data=common_voice_train[rand_int]["audio"]["array"], autoplay=False, rate=16000)

Target text: គីមហ្កិច ជា សាស្ត្រាចារ្យ មួយ រូប ដែល មាន បទពិសោធន៍ ខ្ពស់
Input array shape: (76459,)
Sampling rate: 16000


In [17]:
# This does not prepare the input for the Transformer model.
# This will resample the data and convert the sentence into indices
# Batch here is just for one entry (row)
def prepare_dataset(batch):
    audio = batch["audio"]
    
    # batched output is "un-batched"
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"])
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["sentence"]).input_ids
    return batch

In [18]:
common_voice_train = common_voice_train.map(prepare_dataset, remove_columns=common_voice_train.column_names, num_proc=16)
common_voice_valid = common_voice_valid.map(prepare_dataset, remove_columns=common_voice_valid.column_names, num_proc=16)

In [20]:
# In case the dataset is too long which can lead to OOM. We should filter them out.
# max_input_length_in_sec = 5.0
# common_voice_train = common_voice_train.filter(lambda x: x < max_input_length_in_sec * processor.feature_extractor.sampling_rate, input_columns=["input_length"])

In [19]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt",
        )

        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [20]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [21]:
wer_metric = load_metric("wer")
# cer_metric = load_metric("cer")

In [22]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = tokenizer.pad_token_id

    pred_str = tokenizer.batch_decode(pred_ids)
    label_str = tokenizer.batch_decode(pred.label_ids, group_tokens=False)
    
    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [23]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-xls-r-300m", 
    attention_dropout=0.1,
    layerdrop=0.0,
    feat_proj_dropout=0.0,
    mask_time_prob=0.75, 
    mask_time_length=10,
    mask_feature_prob=0.25,
    mask_feature_length=64,
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer)
)

Some weights of the model checkpoint at facebook/wav2vec2-xls-r-300m were not used when initializing Wav2Vec2ForCTC: ['quantizer.codevectors', 'project_hid.weight', 'project_q.weight', 'project_q.bias', 'quantizer.weight_proj.bias', 'project_hid.bias', 'quantizer.weight_proj.weight']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-xls-r-300m and are newly initialized: ['lm_head.bias', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it 

In [24]:
model.freeze_feature_encoder()

In [25]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir='.',
  group_by_length=True,
  per_device_train_batch_size=8,
  gradient_accumulation_steps=4,
  evaluation_strategy="steps",
  gradient_checkpointing=True,
  fp16=True,
  num_train_epochs=70,
  save_steps=400,
  eval_steps=400,
  logging_steps=100,
  learning_rate=3e-5,
  warmup_steps=1000,
  save_total_limit=3,
  load_best_model_at_end=True
)

In [26]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=common_voice_train,
    eval_dataset=common_voice_valid,
    tokenizer=processor.feature_extractor,
)

Using amp half precision backend


In [27]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length.
/opt/conda/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 2586
  Num Epochs = 70
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 4
  Total optimization steps = 5670


Step,Training Loss,Validation Loss,Wer
400,6.143800,5.319012,1.000000
800,3.611400,3.594915,1.000000
1200,3.509800,3.538626,1.000000
1600,3.382200,3.217674,1.000000
2000,2.374500,1.538638,0.861423
2400,2.019700,1.145986,0.711610
2800,1.851500,0.929940,0.677903
3200,1.727700,0.808237,0.617978
3600,1.657700,0.719269,0.558052
4000,1.575200,0.662438,0.528090


The following columns in the evaluation set  don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length.
***** Running Evaluation *****
  Num examples = 29
  Batch size = 8
Saving model checkpoint to ./checkpoint-400
Configuration saved in ./checkpoint-400/config.json
Model weights saved in ./checkpoint-400/pytorch_model.bin
Configuration saved in ./checkpoint-400/preprocessor_config.json
Deleting older checkpoint [checkpoint-6400] due to args.save_total_limit
The following columns in the evaluation set  don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length.
***** Running Evaluation *****
  Num examples = 29
  Batch size = 8
Saving model checkpoint to ./checkpoint-800
Configuration saved in ./checkpoint-800/config.json
Model weights saved in ./checkpoint-800/pytorch_model.bin
Configuration saved in ./checkpoint-800/preprocessor_config.json
Deleting older checkpoint [checkpoint-6800] due to args.save_

TrainOutput(global_step=5670, training_loss=2.8286902425991376, metrics={'train_runtime': 11640.9835, 'train_samples_per_second': 15.55, 'train_steps_per_second': 0.487, 'total_flos': 2.7418278911692476e+19, 'train_loss': 2.8286902425991376, 'epoch': 70.0})

In [31]:
1

1

In [32]:
tokenizer.push_to_hub('vitouphy/xls-r-300m-km')

tokenizer config file saved in vitouphy/xls-r-300m-km/tokenizer_config.json
Special tokens file saved in vitouphy/xls-r-300m-km/special_tokens_map.json
added tokens file saved in vitouphy/xls-r-300m-km/added_tokens.json
To https://huggingface.co/vitouphy/xls-r-300m-km
   3ef5dfc..cb4f72c  main -> main



'https://huggingface.co/vitouphy/xls-r-300m-km/commit/cb4f72cb420eee8ca1f44b582a9d3cfbcd258f3d'

In [34]:
kwargs = {
    "finetuned_from": "facebook/wav2vec2-xls-r-300m",
    "tasks": "speech-recognition",
    "tags": ["automatic-speech-recognition", "openslr", "robust-speech-event", "km"],
    "dataset_args": f"Config: km, Training split: train, Eval split: validation",
    "dataset": "openslr",
    "language": "km"
}

In [35]:
trainer.create_model_card(**kwargs)

Dropping the following result as it does not have all the necessary fields:
{}


In [36]:
model.push_to_hub('vitouphy/xls-r-300m-km')

Configuration saved in vitouphy/xls-r-300m-km/config.json
Model weights saved in vitouphy/xls-r-300m-km/pytorch_model.bin


Upload file pytorch_model.bin:   0%|          | 3.39k/1.18G [00:00<?, ?B/s]

To https://huggingface.co/vitouphy/xls-r-300m-km
   cb4f72c..8fe8876  main -> main



'https://huggingface.co/vitouphy/xls-r-300m-km/commit/8fe88762a9fca1dce5e056605465042b5700b69e'

In [38]:
trainer.save_model()

Saving model checkpoint to .
Configuration saved in ./config.json
Model weights saved in ./pytorch_model.bin
Configuration saved in ./preprocessor_config.json
